In [14]:
import duckdb
import pandas as pd
from nba_api.stats.endpoints import commonallplayers, leaguedashplayerstats

In [ ]:
# Pull players into a dataframe

players = commonallplayers.CommonAllPlayers(is_only_current_season=0)

df_players = players.get_data_frames()[0]

df_players.head(10)

In [ ]:
# Pull player stats for 2024-2025 season

player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2024-25', per_mode_detailed='PerGame')

df_player_stats = player_stats.get_data_frames()[0]

df_player_stats.head(10)

## Load raw data into duckdb

In [21]:
sql_raw_players_table = """
create or replace table raw.players as
select * from df_players
"""

sql_raw_player_stats_table = """
create or replace table raw.player_stats as 
select * from df_player_stats
"""

with duckdb.connect('../duckdb/nba.duckdb') as con:
    con.execute("create schema if not exists raw")
    con.execute(sql_raw_players_table)
    con.execute(sql_raw_player_stats_table)


In [ ]:
with duckdb.connect('../duckdb/nba.duckdb') as con:
    display(con.sql("""
        select table_schema, table_name
        from information_schema.tables
        order by table_schema, table_name
    """).to_df())
